<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Camera-Calibration" data-toc-modified-id="Camera-Calibration-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Camera Calibration</a></span></li><li><span><a href="#Pipeline" data-toc-modified-id="Pipeline-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Pipeline</a></span><ul class="toc-item"><li><span><a href="#Code" data-toc-modified-id="Code-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Code</a></span><ul class="toc-item"><li><span><a href="#Helper-Functions" data-toc-modified-id="Helper-Functions-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Helper Functions</a></span></li><li><span><a href="#Main-Pipeline" data-toc-modified-id="Main-Pipeline-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Main Pipeline</a></span></li></ul></li></ul></li></ul></div>

# Camera Calibration

* Find camera matrix & distortion coefficients
* Demonstrate undistorting calibration image


In [ ]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
import math

In [ ]:
def calibration_undistort(images):
    '''
    Use images to calculate the camera calibration & image distortion matrices
    '''
    # Object points follow a grid, like (0,0,0), (1,0,0), (2,0,0)...
    objp = np.zeros((9*6,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpts = [] # 3d points in real world space
    imgpts = [] # 2d points in image plane.

    # Step through the list and search for chessboard corners
    for fname in images:
        # Read in each image
        img = cv2.imread(fname)

        # Find the chessboard corners
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # Add object points & image points if found
        if ret == True:
            objpts.append(objp)
            imgpts.append(corners)
     
    # Create matrix using all the images (with corners found)
    # ret, mtx, dist, rvecs, tvecs
    _, mtx, dist, _, _ = cv2.calibrateCamera(objpts, imgpts, img.shape[1:], None, None)
    return (mtx, dist)

In [ ]:
# Load all the images & use them to the matrices 
images = glob.glob('camera_cal/calibration*.jpg')
mtx, dist = calibration_undistort(images)

In [ ]:
# TEST: Plot out each image undistorted 
images = glob.glob('camera_cal/calibration*.jpg')
for fname in images:
    # Undistort the image
    img = cv2.imread(fname)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    
    # Plotting side-by-side
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(undist)
    ax2.set_title('Undistorted and Warped Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
# Save the calibration for later use
pickle.dump({'mtx': mtx, 'dist': dist}, 
            open('camera_cal/calibration_matrices.pickle', 'wb')
)

# Pipeline

For all images:

* Distortion correction image
* Create a thresholded binary image
    - Color transforms, gradients or other methods
* Perspective transform ("birds-eye view")
* Identify lane-line pixels and fit their positions with a polynomial
* Calculate radius of curvature of lane & position of vehicle with respect to center
* Result plotted down onto the road so lane area is identified clearly


## Code

### Helper Functions

In [ ]:
def read_image_from_file(img_filepath, convert=None):
    '''
    Returns the filename/filepath & image (numpy matrix) from given file path.
    '''
    img = cv2.imread(img_filepath)
    
    # Allow for color conversion
    if convert:
        img = cv2.cvtColor(img, convert)
        
    return (img_filepath, img)

def display_images(img_filenames, figsize=(24,8), cols=3, rows=None, convert=None):
    '''
    Display images given a list of images to display. If `rows` is `None`, 
    rows will be calculated given the number of columns `cols`
    '''
    
    # Getting the size to plot; enough rows to fit column
    if not rows:
        rows = math.ceil(len(img_filenames) // cols)
    elif rows*cols < len(img_filenames):
        print(f'Note: not all images will be displayed')
        print(f'{len(img_filenames)} images, but only {rows*cols} will be displayed')
    f, axs = plt.subplots(rows, cols, figsize=figsize)
    
    # Image files and names (using a conversion if needed)
    read_images = lambda fnames: read_image_from_file(fnames, convert=convert)
    fnames_and_imgs = list(map(read_images, img_filenames))
    
    # Iterate over each axis to display the image
    for ax, fname_imgs_tuple in zip(axs.flat, fnames_and_imgs):
        fname, img = fname_imgs_tuple
        ax.set_title(fname)
        ax.imshow(img)
    

### Main Pipeline

In [ ]:
# Load camera calibration matrices from earlier
cameraCalibration = pickle.load( open('camera_cal/calibration_matrices.pickle', 'rb' ) )
mtx, dist = map(cameraCalibration.get, ('mtx', 'dist'))

In [ ]:
# Read in files to convert
test_images = glob.glob('test_images/advance/*.jpg')
# Display the unmodified images
display_images(test_images, cols=4, figsize=(24,8), convert=cv2.COLOR_BGR2RGB)